In [1]:
from tes.data import Data,PacketStream,Packet,Payload,EventFlags
from tes.hdf5 import _PacketData, _import
from tes.base import Detection, Height, Timing, event_dt
import numpy as np
import matplotlib.pyplot as plt
from subprocess import Popen

import logging
import sys
logger = logging.getLogger('tes')
logger.setLevel(logging.DEBUG)
handler = logging.StreamHandler(sys.stderr)
formatter = logging.Formatter('%(name)s:%(levelname)s %(message)s')
handler.setFormatter(formatter)
logger.handlers = [handler]

trace_dt=np.dtype([('raw','i4'), ('f','i4'), ('s','i4')])
flags_dt = (np.int32, {'f0':(np.uint8,0),'f1':(np.uint8,1),'f2':(np.uint8,2), 'f3':(np.uint8,3)})

min_dt=np.dtype([
    ('clk','i4'),('f','i4'),('s','i4'),('low','i4'),('high','i4'),('flags',flags_dt),
    ('max_slope','i4'),('minima','i4'),('s_area','i4'),('s_extrema','i4')
])
max_dt=np.dtype([
    ('clk','i4'),('f','i4'),('s','i4'),('flags',flags_dt),('s_area','i4'),('s_extrema','i4'),
    ('peak_height','i4')
])
f0_xing_dt=np.dtype([
    ('clk','i4'),('f_area','i4'),('f_exrema','i4')
])
ptn_xing_dt=np.dtype([
    ('clk','i4'),('pulse_area','i4'),('pulse_length','i4'),('pulse_timer','i4')
])
rise_start_dt=np.dtype([
    ('clk','i4'),('pulse_timer','i4'),('rise_number','i4'),('rise_address','i4')
])
rise_dt=np.dtype([
    ('clk','i4'),('pulse_timer','i4'),('rise_number','i4'),('rise_address','i4'),
    ('rise_timer','i4'),('peak_height','i4')
])
pulse_start_dt=np.dtype([
    ('clk','i4'),('enabled','i4'),('area_threshold','i4'),('cfd_rel2min','i4'),
    ('constant_fraction','i4'), ('detection','i4'),('height','i4'),('max_peaks','i4'),
    ('pulse_threshold','i4'),('slope_threshold','i4'),('timing','i4'),('trace_type','i4'),
    ('trace_signal','i4'),('trace_length','i4'),('trace_stride','i4')
])
stamp_dt=np.dtype([('clk','i4'),('pulse_timer','i4')])
f0_dt=np.dtype([
    ('rise0','u1'),('rise_start','u1'),('pulse_start','u1'),('will_cross','u1'),('will_arm','u1'),
    ('cfd_error','u1'),('cfd_overrun','u1'),('cfd_valid','u1')
])
f1_dt=np.dtype([
    ('b0','u1'),('b1','u1'),('b2','u1'),('b3','u1'),('b4','u1'),
    ('rise2','u1'),('rise1','u1'),('valid_rise','u1')
])

xing_dt=np.dtype([('clk','i4')])
stream_dt=np.dtype([('clk','i4'),('data','u8')])

def xings(s, thresh):
    xing = np.zeros_like(s)
    i=0
    was_above = s[0] > thresh
    for i in range(1, len(s)):
        if s[i] > thresh:
            if not was_above:
                xing[i] = 1
            was_above = True
        if s[i] < thresh:
            if was_above:
                xing[i] = -1
            was_above = False
            
    return np.where(xing==1)[0], np.where(xing==-1)[0]

In [2]:
traces=Data.fromfile('traces',trace_dt,'teslib','measurement_framer_TB',tool='vivado')
min_data=Data.fromfile('min_data',min_dt,'teslib','measurement_framer_TB',tool='vivado')
max_data=Data.fromfile('max_data',max_dt,'teslib','measurement_framer_TB',tool='vivado')
f0p_xings=Data.fromfile('f0p_xings',f0_xing_dt,'teslib','measurement_framer_TB',tool='vivado')
f0n_xings=Data.fromfile('f0n_xings',f0_xing_dt,'teslib','measurement_framer_TB',tool='vivado')
ptn_xings=Data.fromfile('ptn_xings',ptn_xing_dt,'teslib','measurement_framer_TB',tool='vivado')
rise_starts=Data.fromfile('rise_start',rise_start_dt,'teslib','measurement_framer_TB',tool='vivado')
rise_stops=Data.fromfile('rise_stop',rise_dt,'teslib','measurement_framer_TB',tool='vivado')
height_valids=Data.fromfile('height_valid',rise_dt,'teslib','measurement_framer_TB',tool='vivado')
init_reg=Data.fromfile('init_reg',pulse_start_dt,'teslib','measurement_framer_TB',tool='vivado')
pulse_starts=Data.fromfile('pulse_start',pulse_start_dt,'teslib','measurement_framer_TB',tool='vivado')
stamp_rises=Data.fromfile('stamp_rise',stamp_dt,'teslib','measurement_framer_TB',tool='vivado')
stamp_pulses=Data.fromfile('stamp_pulse',stamp_dt,'teslib','measurement_framer_TB',tool='vivado')
framer_errors=Data.fromfile('framer_error',xing_dt,'teslib','measurement_framer_TB',tool='vivado')
framer_overflows=Data.fromfile('framer_overflow',xing_dt,'teslib','measurement_framer_TB',tool='vivado')
commits=Data.fromfile('commit',xing_dt,'teslib','measurement_framer_TB',tool='vivado')
dumps=Data.fromfile('dump',xing_dt,'teslib','measurement_framer_TB',tool='vivado')
starts=Data.fromfile('start',xing_dt,'teslib','measurement_framer_TB',tool='vivado')
stream=Data.fromfile('stream',stream_dt,'teslib','measurement_framer_TB',tool='vivado')
stream_data=np.copy(stream['data'].byteswap()).view(np.uint8)
stream_lasts=np.where(stream['clk']<0)[0]
raw=traces['raw']
f=traces['f']
s=traces['s']
mins, maxs = xings(s, 0)
f0p, f0n = xings(f,0)

In [3]:
stream_data[:8],stream_lasts[:8]

(array([32,  0,  0,  0, 48,  8,  0,  0], dtype=uint8),
 array([ 3,  7, 11, 15, 19, 23, 27, 31], dtype=int64))

In [31]:
if maxs[0] < mins[0]:
    maxs = maxs[1:] #start with a min
#     max_data=max_data[1:]
if f0n[0] < f0p[0]:
    f0n = f0n[1:]

#get initial registers
reg = init_reg[0]

# flags
min_f0=np.unpackbits(min_data['flags']['f0']).view(f0_dt)
min_f1=np.unpackbits(min_data['flags']['f1']).view(f1_dt)
max_f0=np.unpackbits(max_data['flags']['f0']).view(f0_dt)
max_f1=np.unpackbits(max_data['flags']['f1']).view(f1_dt)

# iterate over minima found by looking at traces
for i in range(len(mins)):
    if mins[i] != min_data['clk'][i]:
        print('index mismatch for minima i={}'.format(i))
        break
    if maxs[i] != max_data['clk'][i]:
        print('index mismatch for maxima i={}'.format(i))
        break
        
    #look for a pulse start at same clk index as this min
    ps = np.where(pulse_starts['clk']==mins[i])[0]
    if len(ps):
        reg = pulse_starts[ps[0]]
        packet_dt = event_dt(reg['detection'], reg['trace_type'],reg['max_peaks'])
        
    maxima = f[maxs[i]]
    minima = f[mins[i]]
    will_cross = maxima > reg['pulse_threshold']
    will_arm = len(np.where(s[mins[i]:maxs[i]+1] > reg['slope_threshold']))!=0
    first_rise = minima < reg['pulse_threshold']
    
    if will_cross != (min_f0['will_cross'][i]==1):
        print('will_cross mismatch @ min i={} '.format(i))
    if will_cross != (max_f0['will_cross'][i]==1):
        print('will_cross mismatch @ max i={}'.format(i))
        
    if will_arm != (min_f0['will_arm'][i]==1):
        print('will_arm mismatch @ min i={} '.format(i))
    if will_arm != (max_f0['will_arm'][i]==1):
        print('will_arm mismatch @ max i={}'.format(i))
    
    if first_rise != (min_f0['rise0'][i]==1):
        print('first_rist mismatch @ min i={}'.format(i))
    if first_rise != (max_f0['rise0'][i]==1):
        print('first_rise mismatch @ max i={}'.format(i))
    
    if first_rise:
        cfi = maxima*reg['constant_fraction']
        cf_min = 0
    else:
        cfi = (maxima-minima)*reg['constant_fraction']
        cf_min = minima
        
    cfi_f,_ = np.modf(cfi)
    cfi_i = np.right_shift(cfi,17)
    cfd_high = maxima - cfi_i
    cfd_low = cf_min + cfi_i

    if min_data['high'][i] != cfd_high:
        print('cfd_high mismatch i={} min_data={} cfd_high={}'.format(i,min_data['high'][i],cfd_high))
    if min_data['low'][i] != cfd_low:
        print('cfd_low mismatch i={} min_data={} cfd_high={}'.format(i,min_data['low'][i],cfd_low))
    cfd_error = cfd_low < minima
    if cfd_error != (min_f0['cfd_error'][i]=='1'):
        print('cfd_error mismatch i={}'.format(i))
    if cfd_error != (min_f0['cfd_valid'][i]=='0'):
        print('cfd_valid mismatch @ min i={}'.format(i))
    if cfd_error != (max_f0['cfd_valid'][i]=='0'):
        print('cfd_valid mismatch @ max i={}'.format(i))
    
    detection=Detection(reg['detection'])
    timing=Timing(reg['timing'])
    height=Height(reg['height'])
    if timing==Timing.cfd_low or height==Height.cfd_high or height==Height.cfd_height:
        valid_rise = will_arm and will_cross and cfd_valid
    else:
        valid_rise = will_arm and will_cross
    if valid_rise != min_f1['valid_rise'][i]:
        print('valid_rise mismatch @ min i={}'.format(i))
    if valid_rise != max_f1['valid_rise'][i]:
        print('valid_rise mismatch @ max i={}'.format(i))
    
    if valid_rise != min_f0['rise_start'][i]==1:
        print('rise_start missing @ min i={}'.format(i))
    if (valid_rise and first_rise) != min_f0['pulse_start'][i]==1:
        print('pulse_start missing @ min i={}'.format(i))
        
    # slope measurements
    if i > 0:
        s_area_min = sum(s[maxs[i-1]:mins[i]+1])/4
        if s_area_min != min_data['s_area'][i]:
            print(
                'slope_area mismatch @ min i={} data={} calc={}'
                .format(i,min_data['s_area'][i],s_area_min)
            )
        s_area_max = sum(s[mins[i-1]:maxs[i]+1])/4
        if s_area_max != max_data['s_area'][i-1]:
            print(
                'slope_area mismatch @ max i={} data={} calc={}'
                .format(i,max_data['s_area'][i-1],s_area_max)
            )
        
    
    
       

slope_area mismatch @ min i=1 data=-1311 calc=-1310.25
slope_area mismatch @ max i=1 data=0 calc=770.5
slope_area mismatch @ min i=2 data=-477 calc=-476.25
slope_area mismatch @ max i=2 data=2082 calc=1846.0
slope_area mismatch @ min i=3 data=-167 calc=-166.5
slope_area mismatch @ max i=3 data=241 calc=144.0
slope_area mismatch @ min i=4 data=-1314 calc=-1313.0
slope_area mismatch @ max i=4 data=71 calc=-1019.75
slope_area mismatch @ min i=5 data=-1401 calc=-1397.75
slope_area mismatch @ max i=5 data=223 calc=1333.5
slope_area mismatch @ min i=6 data=-1496 calc=-1494.25
slope_area mismatch @ max i=6 data=2512 calc=3494.5
slope_area mismatch @ min i=7 data=-1985 calc=-1984.0
slope_area mismatch @ max i=7 data=2478 calc=2269.25
slope_area mismatch @ min i=8 data=-2682 calc=-2679.25
slope_area mismatch @ max i=8 data=1775 calc=583.5
slope_area mismatch @ min i=9 data=-167 calc=-165.75
slope_area mismatch @ max i=9 data=1490 calc=2869.0
slope_area mismatch @ min i=10 data=-2683 calc=-2682.

slope_area mismatch @ max i=238 data=1477 calc=2343.25
slope_area mismatch @ min i=239 data=-1401 calc=-1398.5
slope_area mismatch @ max i=239 data=1952 calc=1602.75
slope_area mismatch @ min i=240 data=-2361 calc=-2360.25
slope_area mismatch @ max i=240 data=1052 calc=532.75
slope_area mismatch @ min i=241 data=-756 calc=-754.75
slope_area mismatch @ max i=241 data=1843 calc=2074.75
slope_area mismatch @ min i=242 data=-294 calc=-293.25
slope_area mismatch @ max i=242 data=988 calc=730.5
slope_area mismatch @ min i=243 data=-1240 calc=-1239.0
slope_area mismatch @ max i=243 data=36 calc=1466.5
slope_area mismatch @ min i=244 data=-2556 calc=-2555.25
slope_area mismatch @ max i=244 data=2671 calc=2230.5
slope_area mismatch @ min i=245 data=-2774 calc=-2772.5
slope_area mismatch @ max i=245 data=2116 calc=210.0
slope_area mismatch @ min i=246 data=-90 calc=-89.75
slope_area mismatch @ max i=246 data=868 calc=2127.25
slope_area mismatch @ min i=247 data=-1310 calc=-1308.75
slope_area mis

slope_area mismatch @ min i=1008 data=-272 calc=-270.5
slope_area mismatch @ max i=1008 data=458 calc=539.25
slope_area mismatch @ min i=1009 data=-797 calc=-795.75
slope_area mismatch @ max i=1009 data=353 calc=1006.0
slope_area mismatch @ min i=1010 data=-2545 calc=-2543.75
slope_area mismatch @ max i=1010 data=1452 calc=1174.75
slope_area mismatch @ min i=1011 data=-243 calc=-242.5
slope_area mismatch @ max i=1011 data=2268 calc=2305.5
slope_area mismatch @ min i=1012 data=-402 calc=-401.25
slope_area mismatch @ max i=1012 data=281 calc=314.5
slope_area mismatch @ min i=1013 data=-1318 calc=-1316.75
slope_area mismatch @ max i=1013 data=435 calc=1816.5
slope_area mismatch @ min i=1014 data=-2610 calc=-2608.75
slope_area mismatch @ max i=1014 data=2700 calc=2664.25
slope_area mismatch @ min i=1015 data=-3642 calc=-3639.75
slope_area mismatch @ max i=1015 data=2574 calc=941.75
slope_area mismatch @ min i=1016 data=-1898 calc=-1897.75
slope_area mismatch @ max i=1016 data=2010 calc=873

slope_area mismatch @ max i=1771 data=1257 calc=1030.75
slope_area mismatch @ min i=1772 data=-686 calc=-684.0
slope_area mismatch @ max i=1772 data=184 calc=1497.0
slope_area mismatch @ min i=1773 data=-2683 calc=-2680.75
slope_area mismatch @ max i=1773 data=1999 calc=1703.5
slope_area mismatch @ min i=1774 data=-1832 calc=-1829.5
slope_area mismatch @ max i=1774 data=2387 calc=2865.75
slope_area mismatch @ min i=1775 data=-2289 calc=-2286.5
slope_area mismatch @ max i=1775 data=2310 calc=2497.75
slope_area mismatch @ min i=1776 data=-139 calc=-137.5
slope_area mismatch @ max i=1776 data=2477 calc=3336.75
slope_area mismatch @ min i=1777 data=-1818 calc=-1814.75
slope_area mismatch @ max i=1777 data=1001 calc=261.0
slope_area mismatch @ min i=1778 data=-2394 calc=-2393.25
slope_area mismatch @ max i=1778 data=1080 calc=920.0
slope_area mismatch @ min i=1779 data=-713 calc=-711.75
slope_area mismatch @ max i=1779 data=2235 calc=1953.25
slope_area mismatch @ min i=1780 data=-2319 calc=

slope_area mismatch @ max i=2515 data=797 calc=1360.5
slope_area mismatch @ min i=2516 data=-3191 calc=-3190.0
slope_area mismatch @ max i=2516 data=1069 calc=1156.0
slope_area mismatch @ min i=2517 data=-1798 calc=-1796.5
slope_area mismatch @ max i=2517 data=3278 calc=1791.0
slope_area mismatch @ min i=2518 data=-2365 calc=-2363.25
slope_area mismatch @ max i=2518 data=312 calc=1114.25
slope_area mismatch @ min i=2519 data=-1571 calc=-1570.25
slope_area mismatch @ max i=2519 data=3168 calc=2556.25
slope_area mismatch @ min i=2520 data=-925 calc=-924.5
slope_area mismatch @ max i=2520 data=959 calc=1786.25
slope_area mismatch @ min i=2521 data=-867 calc=-866.75
slope_area mismatch @ max i=2521 data=1754 calc=1075.5
slope_area mismatch @ min i=2522 data=-135 calc=-134.25
slope_area mismatch @ max i=2522 data=189 calc=536.5
slope_area mismatch @ min i=2523 data=-378 calc=-377.0
slope_area mismatch @ max i=2523 data=482 calc=239.5
slope_area mismatch @ min i=2524 data=-745 calc=-744.5
sl

...

slope_area mismatch @ min i=4772 data=-234 calc=-232.5
slope_area mismatch @ max i=4772 data=513 calc=1804.5
slope_area mismatch @ min i=4773 data=-63 calc=-62.5
slope_area mismatch @ max i=4773 data=1525 calc=3417.25
slope_area mismatch @ min i=4774 data=-4155 calc=-4154.5
slope_area mismatch @ max i=4774 data=1956 calc=-603.25
slope_area mismatch @ min i=4775 data=-941 calc=-940.25
slope_area mismatch @ max i=4775 data=1596 calc=3179.25
slope_area mismatch @ min i=4776 data=-2031 calc=-2030.0
slope_area mismatch @ max i=4776 data=2526 calc=1476.25
slope_area mismatch @ min i=4777 data=-428 calc=-426.25
slope_area mismatch @ max i=4777 data=981 calc=1471.5
slope_area mismatch @ min i=4778 data=-1121 calc=-1119.75
slope_area mismatch @ max i=4778 data=919 calc=433.0
slope_area mismatch @ min i=4779 data=-684 calc=-682.5
slope_area mismatch @ max i=4779 data=636 calc=985.75
slope_area mismatch @ min i=4780 data=-1920 calc=-1919.75
slope_area mismatch @ max i=4780 data=1035 calc=-66.75
s

slope_area mismatch @ max i=5509 data=304 calc=2197.25
slope_area mismatch @ min i=5510 data=-563 calc=-561.75
slope_area mismatch @ max i=5510 data=2954 calc=2692.75
slope_area mismatch @ min i=5511 data=-171 calc=-170.25
slope_area mismatch @ max i=5511 data=301 calc=1048.75
slope_area mismatch @ min i=5512 data=-1283 calc=-1282.5
slope_area mismatch @ max i=5512 data=920 calc=-295.25
slope_area mismatch @ min i=5513 data=-1074 calc=-1073.0
slope_area mismatch @ max i=5513 data=68 calc=73.25
slope_area mismatch @ min i=5514 data=-909 calc=-907.75
slope_area mismatch @ max i=5514 data=1080 calc=1271.75
slope_area mismatch @ min i=5515 data=-430 calc=-429.0
slope_area mismatch @ max i=5515 data=1101 calc=1095.25
slope_area mismatch @ min i=5516 data=-2134 calc=-2133.25
slope_area mismatch @ max i=5516 data=424 calc=-300.75
slope_area mismatch @ min i=5517 data=-1176 calc=-1175.75
slope_area mismatch @ max i=5517 data=1409 calc=2229.5
slope_area mismatch @ min i=5518 data=-1707 calc=-17

slope_area mismatch @ max i=6278 data=806 calc=-354.0
slope_area mismatch @ min i=6279 data=-190 calc=-189.75
slope_area mismatch @ max i=6279 data=655 calc=2584.75
slope_area mismatch @ min i=6280 data=-316 calc=-315.25
slope_area mismatch @ max i=6280 data=2120 calc=2609.25
slope_area mismatch @ min i=6281 data=-344 calc=-343.5
slope_area mismatch @ max i=6281 data=805 calc=462.5
slope_area mismatch @ min i=6282 data=-870 calc=-868.5
slope_area mismatch @ max i=6282 data=1 calc=80.5
slope_area mismatch @ min i=6283 data=-2348 calc=-2347.25
slope_area mismatch @ max i=6283 data=951 calc=-521.5
slope_area mismatch @ min i=6284 data=-31 calc=-30.0
slope_area mismatch @ max i=6284 data=876 calc=1652.75
slope_area mismatch @ min i=6285 data=-522 calc=-521.75
slope_area mismatch @ max i=6285 data=807 calc=593.75
slope_area mismatch @ min i=6286 data=-2254 calc=-2252.75
slope_area mismatch @ max i=6286 data=309 calc=1508.75
slope_area mismatch @ min i=6287 data=-1998 calc=-1995.5
slope_area

slope_area mismatch @ min i=7032 data=-53 calc=-51.5
slope_area mismatch @ max i=7032 data=2485 calc=4273.75
slope_area mismatch @ min i=7033 data=-4569 calc=-4568.25
slope_area mismatch @ max i=7033 data=1842 calc=-1189.25
slope_area mismatch @ min i=7034 data=-469 calc=-468.75
slope_area mismatch @ max i=7034 data=1539 calc=3379.75
slope_area mismatch @ min i=7035 data=-2620 calc=-2618.5
slope_area mismatch @ max i=7035 data=2310 calc=1394.25
slope_area mismatch @ min i=7036 data=-562 calc=-561.25
slope_area mismatch @ max i=7036 data=1703 calc=1377.5
slope_area mismatch @ min i=7037 data=-584 calc=-582.75
slope_area mismatch @ max i=7037 data=236 calc=481.25
slope_area mismatch @ min i=7038 data=-1851 calc=-1849.25
slope_area mismatch @ max i=7038 data=830 calc=115.0
slope_area mismatch @ min i=7039 data=-510 calc=-508.0
slope_area mismatch @ max i=7039 data=1137 calc=1502.0
slope_area mismatch @ min i=7040 data=-1189 calc=-1186.25
slope_area mismatch @ max i=7040 data=876 calc=1950

IndexError: index 7640 is out of bounds for axis 0 with size 7640

In [24]:
j=1
s[mins[j]:maxs[j]]

array([  2,   3,   6,   8,  11,  13,  15,  17,  17,  17,  17,  17,  17,
        18,  20,  23,  25,  28,  31,  32,  34,  36,  38,  41,  44,  47,
        49,  52,  53,  55,  57,  60,  63,  66,  70,  73,  77,  80,  84,
        88,  93,  97, 101, 103, 106, 110, 114, 118, 123, 126, 129, 132,
       134, 137, 140, 142, 143, 145, 146, 148, 150, 151, 152, 152, 152,
       151, 151, 151, 150, 149, 148, 147, 145, 143, 140, 136, 131, 127,
       122, 119, 115, 112, 109, 105, 100,  96,  91,  85,  80,  75,  69,
        64,  59,  53,  47,  42,  36,  31,  27,  23,  19,  16,  11,   5,   0])

In [25]:
mins[1]

1116

In [ ]:
j=2
mins[j],min_data['clk'][j],maxs[j],max_data['clk'][j]

In [ ]:
init_reg=Data.fromfile('init_reg',pulse_start_dt,'teslib','measurement_framer_TB',tool='vivado')
init_reg['pulse_threshold']

In [ ]:
pulse_starts['clk']

In [ ]:
min_data['f'][i],f[mins[i]:mins[i]+10]

In [ ]:
min_data['s'][i]

In [ ]:
min_data['clk'][i],mins[i]

In [ ]:
%matplotlib notebook
for k in range(1, 1000):
#     plt.plot([0,l],[0,0],':k')
#     plt.plot(s[mins[k]:maxs[k]+1],'r')
    plt.plot(f[mins[k]:maxs[k]+1],'b')